In [ ]:
import confmain
import os
import time
import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow.keras.regularizers import L1L2

from util_module import util_func
from util_module import model_func

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

#### **Custom Model**

In [ ]:
def generate_custom_model(input_shape, output, lr=1e-3, n_layer=1):
    # optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    optimizer = tf.keras.optimizers.RMSprop(learning_rate=lr)

    input_layer = tf.keras.layers.Input(shape=input_shape)

    x = tf.keras.layers.Conv1D(8, 5, padding="same")(input_layer)
    x = tf.keras.layers.LayerNormalization()(x)
    x = tf.keras.layers.ReLU()(x)

    x = tf.keras.layers.Conv1D(16, 5, padding="same")(x)
    x = tf.keras.layers.LayerNormalization()(x)
    x = tf.keras.layers.ReLU()(x)

    x = tf.keras.layers.Conv1D(32, 5, padding="same")(x)
    x = tf.keras.layers.LayerNormalization()(x)
    x = tf.keras.layers.ReLU()(x)

    x = tf.keras.layers.Conv1D(64, 5, padding="same")(x)
    x = tf.keras.layers.LayerNormalization()(x)
    x = tf.keras.layers.ReLU()(x)

    x = tf.keras.layers.Conv1D(128, 5, padding="same")(x)
    x = tf.keras.layers.LayerNormalization()(x)
    x = tf.keras.layers.ReLU()(x)

    x = tf.keras.layers.Conv1D(
        256, 7, padding="same", kernel_regularizer=tf.keras.regularizers.l2(1e-4)
    )(x)
    x = tf.keras.layers.LayerNormalization()(x)
    x = tf.keras.layers.ReLU()(x)

    x = tf.keras.layers.Conv1D(
        512, 7, padding="same", kernel_regularizer=tf.keras.regularizers.l2(1e-4)
    )(x)
    x = tf.keras.layers.LayerNormalization()(x)
    x = tf.keras.layers.ReLU()(x)

    x = tf.keras.layers.Dropout(0.3)(x)
    for _ in range(n_layer):
        x = tf.keras.layers.Bidirectional(
            tf.keras.layers.LSTM(
                input_shape[0],
                return_sequences=True,
                kernel_regularizer=tf.keras.regularizers.l2(1e-4),
            )
        )(x)

    x = tf.keras.layers.Dense(512, activation="relu")(x)
    x = tf.keras.layers.Dropout(0.3)(x)
    output_layer = tf.keras.layers.Dense(
        output, activation="softmax", kernel_regularizer=tf.keras.regularizers.l2(1e-4)
    )(x)

    model = tf.keras.models.Model(inputs=input_layer, outputs=output_layer)
    model.compile(
        optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"]
    )
    return model

#### **CNN-BiGRU**

In [ ]:
def generate_cnn_bigru(input_shape, output, lr=1e-3):
    optimizer = tf.keras.optimizers.RMSprop(learning_rate=lr)

    input_layer = tf.keras.layers.Input(shape=input_shape)

    x = tf.keras.layers.Conv1D(8, 1, padding="same", activation='relu')(input_layer)
    x = tf.keras.layers.Conv1D(16, 1, padding="same", activation='relu')(x)
    x = tf.keras.layers.Conv1D(32, 1, padding="same", activation='relu')(x)
    x = tf.keras.layers.Conv1D(64, 1, padding="same", activation='relu')(x)
    x = tf.keras.layers.Conv1D(128, 1, padding="same", activation='relu')(x)
    x = tf.keras.layers.Bidirectional(
        tf.keras.layers.GRU(512, return_sequences=True, activation='tanh')
    )(x)

    output_layer = tf.keras.layers.Dense(output, activation="softmax")(x)

    model = tf.keras.models.Model(inputs=input_layer, outputs=output_layer)
    model.compile(
        optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"]
    )

    return model

In [ ]:
def model_training_wrapper(lead, model_info):
    train_set, val_set, test_set, _ = util_func.get_x_y(f'../data/ludb_processed/ludb_{lead}.pickle')

    X_train, y_train = train_set
    X_val, y_val = val_set
    X_test, y_test = test_set

    # model = generate_cnn_bigru((816, 1), 8, 0.001)
    model = generate_custom_model((816, 1), 8, 1e-3, 1) # PPROPOSED!

    start_time = time.time()
    history = model.fit(X_train, y_train, epochs=model_info['epochs'], batch_size=model_info['batch_size'], validation_data=(X_val, y_val))
    stop_time = time.time()

    model_info['Time elapsed'] = stop_time - start_time

    model_func.generate_results(model, history.history, model_info, train_set, val_set, test_set)

In [ ]:
LEADS = ['i', 'ii', 'iii', 'avr', 'avl', 'avf', 'v1', 'v2', 'v3', 'v4', 'v5', 'v6']
# LEADS = ['avf']
# LEADS = ['v1', 'v2', 'v3', 'v4', 'v5', 'v6']

for lead in LEADS:
    model_info = {
        'name': f'{lead}-CustomModel',
        'lead': lead,
        'batch_size': 32,
        'epochs': 300,
        'n_layer': 1,
        'optimizer': 'RMSprop(lr=0.001)',
        'additional info': ''
    }

    try:
        model_training_wrapper(lead, model_info)
    
    except:
        print(f'Lead-{lead} failed.')